# Playground for ASFPy module development

In [47]:
import os
import csv
import random
import operator
from copy import deepcopy

from asfpy import asfpy

In [48]:
from pathlib import Path

FOLDER = "asfp2020"
EDITORS_FILE = "processed-editors-list-draft.csv"
APPLICANTS_FILE = "processed-applicants-list-draft.csv"
MATCHINGS_FILE = "matchings-list-draft.csv"

path = Path("__file__").parent.absolute()

editors_filename = path.parent.joinpath(FOLDER).joinpath(EDITORS_FILE)
editors = asfpy.read_preprocessed_editors_list_csv(editors_filename)

applicants_filename = path.parent.joinpath(FOLDER).joinpath(APPLICANTS_FILE)
applicants = asfpy.read_preprocessed_applicants_list_csv(applicants_filename)

matchings_filename = path.parent.joinpath(FOLDER).joinpath(MATCHINGS_FILE)

In [51]:
_editors = deepcopy(editors)
_applicants = deepcopy(asfpy.randomize(applicants))

p_applicants = asfpy.prioritize(_applicants, rank_method = asfpy.asfp_rank)

allocation_output = asfpy.allocate(p_applicants, _editors)

# Allocation Output Handling

The output from running the allocation method includes a list of matchings, a list of unmatched applicants by identifier strings, and a list of editors _after_ decrementing capacities.

## Handle saving matchings

In [52]:
matchings = allocation_output["matchings"]
dyads = asfpy.format_matchings(matchings, applicants, editors)

asfpy.write_list_to_csv(dyads, matchings_filename)

print("Number of matchings: ", len(allocation_output["matchings"]))
print("Number of unmatched: ", len(allocation_output["unmatched"]))
print("Capacity left by editors: ", asfpy.capacity(allocation_output["editors"]))

Number of matchings:  97
Number of unmatched:  41
Capacity left by editors:  34


## New editors by capacity and unmatched applicants

This pattern should be used to re-run the allocation for a given set of editors and applicants. In particular, this would be re-run _if_ editors increase capacity. The applicants remain in the same priority ordering from the first allocation by using `p_applicants`.

In [56]:
new_capacity_editors = allocation_output["editors"]
unmatched_ids = allocation_output["unmatched"]
## Use p_applicants to keep rank order
unmatched_applicants = asfpy.compile_unmatched(unmatched_ids, p_applicants)

asfpy.allocate(unmatched_applicants, new_capacity_editors)["matchings"]

[]